In [1]:
# Lo primero, como siempre, es el curre (tedioso a veces) de preparar el dataframe para el modelado. Pero no desesperes,
 #en unas pocas líneas de código estaremos abordando el verdadero problema: cómo estimar el voto a partir de datos de 
 #una encuesta con un modelito de ML.

# Importamos librerías y cargamos el dataframe desde la url de descarga del "Sociómtero Vasco 83, 
 #Elecciones Autonómicas: previsión de voto".

import numpy as np
import pandas as pd

df = pd.read_csv("https://www.euskadi.eus/contenidos/ds_informes_estudios/od_24sv83/opendata/24sv83.csv", sep = ";")

# Realizamos un análisis exploratorio básico y comprobamos, tal y como era de esperar, que no hay NA's. Los 
 #profesionales del Sociómetro ya se han encargado de limpiar la BBDD por nosotros.

df.head()
df.describe()
df.isna().sum().sum()

# Además, el análisis exploratorio muestra cómo 79 de 80 variables son de tipo numérico.
# En realidad muchas de estas variables son categóricas y esos números son los códigos asignados a esas categorías 
 #en el cuestionario.
# Total, que ahora toca un trabajito pelín artesano/manual, mirar el cuestionario y ver qué preguntas/variables son
 #categóricas y cuáles son realmente numéricas. Esta discriminación es importante para el entrenamiento de nuestro 
 #modelo de ML.

df.dtypes.value_counts()

int64     79
object     1
dtype: int64

In [2]:
# Si estás siguiendo este script es importante que mires el cuestionario para entender cómo hemos discriminado entre
 #variables categóricas y numéricas. Aquí abajo tienes el link.
"https://www.euskadi.eus/contenidos/ds_informes_estudios/od_24sv83/es_def/adjuntos/24sv83_gal.pdf"
 #Hacia la mitad del pdf está la versión en castellano.

# Vamos a ello, después de hacer un barrido al cuestionario, ya hemos identificado qué variables son categóricas y cuáles
 #son realmente numéricas. Hemos incluido las variables ordinales como numéricas, entendiendo que operan como una variable
 #discreta y que su procesamiento como variable numérica es más apropiado para modelar.
# Listamos únicamente las variables categóricas ya que las numéricas/ordinales se quedarán con el dtype original ("object").

categorical_vars = ['lurral', 'tfno', 'P0A', 'P0B', 'P01', 'P03', 'P18', 'P20', 'P21', 'P23', 'P2401', 'P2402', 'P2403', 
                    'P2501', 'P2502', 'P2503', 'P27A01', 'P27A02', 'P27A03', 'P27A04', 'P27A05', 'P27A06', 'P27A07',
                    'P35', 'P36', 'P37', 'P38', 'P39']

# Además, hemos identificado 3 variables que no aportarán nada al modelo: 'elkar', 'inkes' y 'wt' que son número 
 #de entrevista, número de encuestador y una variable interna de Sociómetro, respectivamente.

vars_to_drop = ['elkar', 'inkes', 'wt']

# Ahora sí, cambiamos el tipo de variable para nuestras variables categóricas.

df[categorical_vars] = df[categorical_vars].astype(object)

# Y eliminamos las variables que no aportarán al modelo.

df.drop(vars_to_drop, axis = 1, inplace = True)

# Comprobamos que el cambio de tipo de variables y el droppeo se han realizado

df.columns
df.dtypes.value_counts()

int64     49
object    28
dtype: int64

In [3]:
# Vale, lo que viene ahora tiene su complejidad, pero creo que se entiende. Tenemos un problema añadido con las variables
 #que están tipificadas como numéricas. La mayoría de ellas no son numéricas puras (como 'P0B' que es la edad, por 
 #ejemplo) sino que son ordinales, esto es, sus categorías de respuesta son un gradiente que sigue una escala que puede
 #ser representada numéricamente. Por ejemplo, la 'P04' es una pregunta sobre salud general del encuestado cuyas posibles 
 #respuestas son 'muy buena','buena', 'regular', 'mala' y 'muy mala' que han sido codificadas del 1 al 5 respectivamente.
 #Hasta aquí bien, tiene sentido pese a ser una escala inversa (el 1 es 'muy buena' y el 5 'muy mala', pero para nuestro
 #modelo funcionará). ¿Entonces, cuál es el problema? Que hay una sexta categoría de respuesta "No sabe-No contesta"
 #codificada con el 6. Y "No sabe-No contesta" (6) no es una peor salud que "muy mala" (5), así que su codificación como 6 
 #puede llevar a nuestro modelo a error. Total, lo que debemos hacer es conseguir que esos "No sabe-No contesta", que no 
 #aportan información relevante, se codifiquen de forma neutra para que no confundan al modelo. ¿Cómo lo hacemos? Pues 
 #codificando esos "No sabe-No contesta" no como 6 sino como el promedio de respuestas (de "muy mala" a "muy buena", es 
 #decir, de 1 a 5) de esa variable. Ese promedio será neutro y no confundirá al modelo.
    
# Y podrás pensar, vale, pues para eso tipificamos las variables como categóricas y ya está. Sí pero no en este caso.
 #Sería la solución más fácil, y probablemente la más correcta. Pero en este caso tenemos un problema, al tratarse de una
 #encuesta el número de casos es muy limitado y convertir todas las variables a categóricas dispararía la dimensionalidad 
 #(una variable por categoría de respuesta) pudiendo tener casi igual número de variables que de casos. Y en ese escenario
 #el modelo no dará buenas prestaciones. Así que no es una opción, cuantas más numéricas/ordinales mejor. A recodificar.

# Después de este rollazo. Pues eso, que vamos a recodificar las variables numéricas para que las respuestas tipo "No sabe",
 #"No contesta", "Resto"...etc tengan como código numérico el promedio de las respuestas de esa variable para que sean 
 #neutras de cara al modelo. Y ya está, lo que hacen las líneas de código de debajo es eso, no te rompas la cabeza de más.

numerical_3cat = ['P1001', 'P1002', 'P11', 'P16', 'P22', 'P31', 'P34']
numerical_4cat = ['P05', 'P06', 'P07', 'P08', 'P09', 'P13', 'P17', 'P19']
numerical_5cat = ['P04', 'P1401', 'P1402', 'P1403', 'P1404', 'P1405', 'P1501', 'P1502', 'P1503', 'P1504', 'P1505', 'P30',
                 'P32', 'P33']
numerical_10cat = ['P1201', 'P1202', 'P1203', 'P2601', 'P2602', 'P2603', 'P2604', 'P2605', 'P2606', 'P2607', 'P27B01', 
                   'P27B02', 'P27B03', 'P27B04', 'P27B05', 'P27B06', 'P27B07', 'P28', 'P29']

for var in numerical_3cat:
    df.loc[df[var] > 3, var] = df[var].mean()
for var in numerical_4cat:
    df.loc[df[var] > 4, var] = df[var].mean()
for var in numerical_5cat:
    df.loc[df[var] > 5, var] = df[var].mean()
for var in numerical_10cat:
    df.loc[df[var] > 10, var] = df[var].mean()

In [4]:
# Vale, ya hemos hecho buena parte del preprocesado. Seguimos con el preprocesado de las dos variables objetivo de cara a 
 #la estimación de voto y nuestro modelo de ML: intención de voto en las próximas elecciones y recuerdo de voto en las 
 #elecciones pasadas, que en el cuestionario/dataframe son la 'P21' y la 'P23' resèctivamente.
 #Y lo primero que vamos a hacer es, ya que son variables categóricas con las que vamos a trabajar de cara a la estimación,
 #recodificar esas variables para que podamos ver la respuesta/categoría y no el código asociado a la misma. Toca volver a
 #echar vistacín al cuestionario, vuelta al trabajito artesano/manual. Ahí va esa recodificación.

print(f"Categorías P21 antes de recode: {df['P21'].unique()}")
print(f"Categorías P23 antes de recode: {df['P23'].unique()}")

p21_dict = {1:'PNV/EAJ', 2:'EH BILDU', 3:'PSE-EE', 4:'PODEMOS', 5:'SUMAR', 6:'PP', 7:'VOX', 8:'OTRO',
           9: 'NO VOTARÁ', 10:'VOTO BLANCO', 11:'VOTO NULO', 12:'NO SABE', 13:'NO CONTESTA', 14:'RESTO'}
p23_dict = {1:'PNV/EAJ', 2:'EH BILDU', 3:'PSE-EE', 4:'PODEMOS', 5: 'PP + CIUDADANOS', 6:'VOX', 7:'OTRO',
            8:'NO VOTÓ', 9:'VOTÓ EN BLANCO', 10:'VOTÓ NULO', 11:'NO PUDO VOTAR POR NO TENER DERECHO',
            12:'NO PUDO VOTAR POR SER MENOR DE EDAD', 13:'NO SABE', 14: 'NO CONTESTA'}

df['P21'] = df['P21'].replace(p21_dict)
df['P23'] = df['P23'].replace(p23_dict)

print('\n')
print(f"Categorías P21 después de recode: {df['P21'].unique()}")
print(f"Categorías P23 después de recode: {df['P23'].unique()}")

Categorías P21 antes de recode: [12 1 13 14 3 6 2 7 11 4 5 10 8 9]
Categorías P23 antes de recode: [4 1 14 3 13 5 2 8 12 6 11 9 7 10]


Categorías P21 después de recode: ['NO SABE' 'PNV/EAJ' 'NO CONTESTA' 'RESTO' 'PSE-EE' 'PP' 'EH BILDU' 'VOX'
 'VOTO NULO' 'PODEMOS' 'SUMAR' 'VOTO BLANCO' 'OTRO' 'NO VOTARÁ']
Categorías P23 después de recode: ['PODEMOS' 'PNV/EAJ' 'NO CONTESTA' 'PSE-EE' 'NO SABE' 'PP + CIUDADANOS'
 'EH BILDU' 'NO VOTÓ' 'NO PUDO VOTAR POR SER MENOR DE EDAD' 'VOX'
 'NO PUDO VOTAR POR NO TENER DERECHO' 'VOTÓ EN BLANCO' 'OTRO' 'VOTÓ NULO']


In [5]:
# Venga, y ya lo ultimito antes de ponernos con el modelito de ML y explicar cómo se estima el voto. Tanto 'P21' como 'P23' 
 #tienen categorías que, a efectos de estimar voto, son similares. Vamos a recodificar de nuevo. En 'P21' vamos a unificar
 #'NO VOTARÁ' y 'RESTO' como 'NO VOTARÁ', y 'NO SABE' y 'NO CONTESTA' como 'NS/NC'.
 #En 'P23' vamos a unificar 'NO VOTÓ', 'NO PUDO VOTAR POR NO TENER DERECHO' y 'NO PUDO VOTAR POR SER MENOR DE EDAD' como
 #'NO VOTÓ', y 'NO SABE' y 'NO CONTESTA' como 'NS/NC'.

p21_dict = {'RESTO': 'NO VOTARÁ', 'NO SABE': 'NS/NC', 'NO CONTESTA':'NS/NC'}
p23_dict = {'NO PUDO VOTAR POR NO TENER DERECHO': 'NO VOTÓ', 'NO PUDO VOTAR POR SER MENOR DE EDAD': 'NO VOTÓ',
            'NO SABE': 'NS/NC', 'NO CONTESTA': 'NS/NC'}

df['P21'] = df['P21'].replace(p21_dict)
df['P23'] = df['P23'].replace(p23_dict)

print(f"Categorías P21 después de último recode: {df['P21'].unique()}")
print(f"Categorías P23 después de último recode: {df['P23'].unique()}")

# Y ahora sí. ¿Sabéis cuál es el principal problema que encontramos los que nos dedicamos, o nos hemos dedicado a esto, para
 #estimar el voto? Oh, sorpresa, que hay mucho (pero mucho, 1220 casos) encuestado que no te dice a quién va a votar.
    
print('\n')
print(df['P21'].value_counts())

# Total, que vamos a entrenar un modelo de ML capaz de predecir la intención de voto de 'P21' a partir del resto de
 #variables. De modo que nuestra variable objetivo será 'P21', nuestros conjuntos de train y test serán todos los casos 
 #distintos de 'NS/NC' en 'P21' y nuestro conjunto de pred (a predecir) todos los casos que son 'NS/NC' en 'P21'.
 #¿Se entiende, no? Y con la salida del predict sustituiremos los 1220 casos de 'NS/NC' y, voila, ya tendremos datos de 
 #intención de voto para todos los casos.
 #Y sí, es un problema peculiar, porque hemos hecho bastante preprocesado previo al split y los conjuntos de train&test y
 #pred tienen un tamaño muy similar, peeerooo no hay manera de hacerlo de otro modo.

Categorías P21 después de último recode: ['NS/NC' 'PNV/EAJ' 'NO VOTARÁ' 'PSE-EE' 'PP' 'EH BILDU' 'VOX' 'VOTO NULO'
 'PODEMOS' 'SUMAR' 'VOTO BLANCO' 'OTRO']
Categorías P23 después de último recode: ['PODEMOS' 'PNV/EAJ' 'NS/NC' 'PSE-EE' 'PP + CIUDADANOS' 'EH BILDU'
 'NO VOTÓ' 'VOX' 'VOTÓ EN BLANCO' 'OTRO' 'VOTÓ NULO']


NS/NC          1220
EH BILDU        571
PNV/EAJ         550
NO VOTARÁ       361
PSE-EE          152
PP               54
VOTO BLANCO      29
PODEMOS          26
SUMAR            26
VOX              15
VOTO NULO        13
OTRO             13
Name: P21, dtype: int64


In [6]:
# Importamos librerías que vamos a necesitar en este bloque.

from sklearn.model_selection import train_test_split

# Ahora sí, vamos con el modelo de ML. Atendiendo a los criterios apuntados justo arriba, definimos los conjuntos de 
 #train&test y pred. Guardamos el conjunto de pred y trabajamos sobre el de train&test.

df_train_test = df[df['P21'] != 'NS/NC']
df_pred = df[df['P21'] == 'NS/NC']

# Definimos variables predictoras (X) y variable objetivo (y) para el conjunto de train&test.

X = df_train_test.drop('P21', axis = 1)
y = df_train_test['P21']

# Hacemos el split para obtener los conjuntos de train y test. Definimos un conjunto de test pequeño para optimizar al
 #máximo el entrenamiento.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

# Vamos a preparar nuestro conjunto de train para el fit del modelo de ML. Convertimos a one-hot las variables categóricas 
 #de nuestro X_train.

categorical_vars = [col for col in X_train.columns if X_train[col].dtype == 'object']
X_train = pd.get_dummies(X_train, columns=categorical_vars)

# Añadimos una categoría "_Others" a cada una de las variables categóricas. Como veremos más abajo servirá para garantizar 
 #presencia de las mismas variables en train y test, asimilando ambos conjuntos. Anótate esta porque es buena, y te resuelve
 #un problema que es más habitual de lo que creemos.

for var in categorical_vars:
    X_train[var+"_Others"] = 0

C:\Users\Victor\anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [7]:
# Repetimos la última transformación del bloque anterior para X_test. Convertimos a one-hot las variables categóricas. 

X_test = pd.get_dummies(X_test, columns=categorical_vars)

C:\Users\Victor\anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [8]:
# Apoyándonos en las categorías "var+_Others" (añadidas más arriba) creamos, eliminamos y transformamos variables para 
 #asimilar los conjuntos de train y test. Lo dicho antes, anótatela porque es buena y resuelve.

vars_no_incommon_train = [var for var in X_train.columns if var not in X_test.columns]
vars_no_incommon_test = [var for var in X_test.columns if var not in X_train.columns]

for var in vars_no_incommon_train:
    X_test[var] = 0
for var in vars_no_incommon_test:
    X_test = X_test.drop(var, axis = 1)
    X_test[var+"_Others"] = 1

X_train_columnssorted = sorted(X_train.columns)
X_train = X_train[X_train_columnssorted]
X_test = X_test[X_train_columnssorted]

# Me guardo un copia de X_train para más adelante

X_train_copy = X_train

# Comprobamos los shape de X_train y X_test para corroborar que todo está OK.

print(f'shape de X_train: {X_train.shape} | shape de X_test: {X_test.shape}')

shape de X_train: (1448, 272) | shape de X_test: (362, 272)


In [9]:
# Importamos librerías que vamos a necesitar en este bloque.

from sklearn.preprocessing import StandardScaler

# Volvemos al conjunto de train. Normalizamos las variables numéricas en X_train.

variables_to_norm = [col for col in X_train.columns if X_train[col].dtype == 'int64']

scaler = StandardScaler()
scaler.fit(X_train[variables_to_norm])

X_train[variables_to_norm] = scaler.transform(X_train[variables_to_norm])

# Transformamos X_train en arreglo numpy para poder entrenar nuestro modelo de ML.

X_train = X_train.values

In [10]:
# Repetimos las transformaciones del bloque anterior para X_test. Importante, la normalización con el scaler de train.

# Normalizamos las variables numéricas en X_test.

X_test[variables_to_norm] = scaler.transform(X_test[variables_to_norm])

# Transformamos X_test en arreglo numpy para poder entrenar nuestro modelo de ML.

X_test = X_test.values

# Y ya está, todo listo para nuestro modelo de ML.

In [11]:
# Importamos librerías necesarias para este bloque

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Vamos a entrenar nuestro modelo de ML. Random Forest suele dar buenas prestaciones en este tipo de problemas de
 #clasificación. Utilizaremos técnicas de grid para optimizar algunos de los hiperparámetros y afinar el modelo.
    
# Instanciamos nuestro modelo (el parámetro 'class_weight' corregirá el desbalanceo de categorías a predecir).

rf_classifier = RandomForestClassifier(class_weight='balanced', random_state=17)

# Definimos los hiperparámetros a optimizar.

param_grid = {'n_estimators': [100, 200, 300],'max_depth': [None, 10, 25, 50]}

# Creamos el objeto de grid y entrenamos nuestro modelo.

grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Visualizamos prestaciones del modelo sobre el conjunto de test.

y_predicted = grid_search.predict(X_test)
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

    EH BILDU       0.95      0.93      0.94       112
   NO VOTARÁ       0.91      1.00      0.95        87
        OTRO       0.00      0.00      0.00         2
     PNV/EAJ       0.90      0.95      0.93       104
     PODEMOS       0.67      0.33      0.44         6
          PP       1.00      0.78      0.88         9
      PSE-EE       0.73      0.92      0.81        26
       SUMAR       1.00      0.29      0.44         7
 VOTO BLANCO       1.00      0.20      0.33         5
   VOTO NULO       0.00      0.00      0.00         3
         VOX       1.00      1.00      1.00         1

    accuracy                           0.90       362
   macro avg       0.74      0.58      0.61       362
weighted avg       0.90      0.90      0.89       362



C:\Users\Victor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Victor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Victor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
# El modelo alcanza un accuracy global del 90%, acertando especialmente en el voto a las
 #opciones mayoritarias (y es normal, ha podido entrenar con más casos de estas opciones). Falla ligeramente o 
 #infrarepresenta con Podemos, Sumar, voto blanco, voto nulo y Vox. Con todo son unas métricas bastante aceptables.

In [13]:
# Quizá podamos pensar que el modelo puede mejorar con otros hiperparámetros. Visualizamos rápidamente las prestaciones 
 #para los distintos hiperparámetros probados.

results_grid = pd.DataFrame(grid_search.cv_results_)

relevant_columns = ['param_n_estimators', 'param_max_depth', 'mean_test_score']
results_grid = results_grid[relevant_columns]

print(results_grid)

   param_n_estimators param_max_depth  mean_test_score
0                 100            None         0.883983
1                 200            None         0.883985
2                 300            None         0.884677
3                 100              10         0.892951
4                 200              10         0.895717
5                 300              10         0.893645
6                 100              25         0.883293
7                 200              25         0.884675
8                 300              25         0.884677
9                 100              50         0.883983
10                200              50         0.883985
11                300              50         0.884677


In [14]:
# Observamos que las prestaciones del modelo varían mínimamente de unos hiperparámetros a otros, y que a partir de una
 #profundidad de 25 las variaciones son residuales. Total, tenemos hiperparámetros los óptimos y un modelo razonablemente
 #bueno.

In [15]:
# Vale, pues ahora sí que sí, vamos a ejecutar el modelo sobre el conjunto pred para predecir las opciones de voto que
 #sustituirán a los 1220 NS/NC.
# Lo primero que tenemos que hacer es aplicar sobre el conjunto pred (haremos una copia llamada 'df_for_pred') todas las 
 #transformaciones que hicimos sobre el conjunto de train (recordad que guardé una copia llamada 'X_train_copy'), para 
 #garantizar mismo formato y características en ambos conjuntos.

# Eliminamos la variable objetivo/a predecir.

df_for_pred = df_pred.drop('P21', axis = 1)

# Convertimos a one-hot las variables categóricas.

df_for_pred = pd.get_dummies(df_for_pred, columns=categorical_vars)

# Añadimos una categoría "_Others" a cada una de las variables categóricas.

for var in categorical_vars:
    df_for_pred[var+"_Others"] = 0

# Apoyándonos en las categorías "var+_Others" creamos, eliminamos y transformamos variables para 
 #asimilar los conjuntos de train y pred.

vars_no_incommon_train = [var for var in X_train_copy.columns if var not in df_for_pred.columns]
vars_no_incommon_pred = [var for var in df_for_pred.columns if var not in X_train_copy.columns]

for var in vars_no_incommon_train:
    df_for_pred[var] = 0
for var in vars_no_incommon_pred:
    df_for_pred = df_for_pred.drop(var, axis = 1)
    df_for_pred[var+"_Others"] = 1

df_for_pred = df_for_pred[X_train_columnssorted]

# Comprobamos los shape de X_train_copy y df_for_pred para corroborar que todo está OK.

print(f'shape de X_train_copy: {X_train_copy.shape} | shape de df_pred: {df_for_pred.shape}')

C:\Users\Victor\anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


shape de X_train_copy: (1448, 272) | shape de df_pred: (1220, 272)


In [16]:
# Comprobados los shape, todo OK. Seguimos con las últimas transformaciones.
# Normalizamos las variables numéricas en df_pred.

df_for_pred[variables_to_norm] = scaler.transform(df_for_pred[variables_to_norm])

# Transformamos df_pred en arreglo numpy para poder entrenar nuestro modelo de ML.

df_for_pred = df_for_pred.values

# Y ya está, todo listo para ejecutar el predict. Vamos a ello.

y_predicted = grid_search.predict(df_for_pred)

# Observamos cómo ha ido el predict.

opciones, conteos = np.unique(y_predicted, return_counts=True)
for opcion, conteo in zip(opciones, conteos):
    print(f'{opcion}:{conteo}')

EH BILDU:106
NO VOTARÁ:762
PNV/EAJ:240
PODEMOS:2
PP:13
PSE-EE:67
SUMAR:1
VOTO BLANCO:29


In [17]:
# Bueno, pues solo nos queda sustituir los nuevos datos de la salida del predict. Vaya, tenemos que sustituir
#los 'NS/NC' del conjunto original de pred por nuestro y_predicted. Y ya estaría. O casi.
#A modo de recordatorio, que ya a estas alturas nos habremos perdido. Los dos subconjuntos originales que tenemos limpitos
#y sin cacharreo más allá de los recodes y renombrados de categorías de respuesta son 'df_train_test' y 'df_pred' (los
#tienes al inicio de la celda 6). Después de sustituir los 'NS/NC' haremos un concat y ya estaría. O casi.

df_pred['P21'] = y_predicted

df_all = pd.concat([df_train_test, df_pred])

#Y después de todo esto...volvemos a visualizar 'P21' y, magia, los 1220 'NS/NC' han desparecido (han sido sustituidos por
 #nuestras predicciones) y ya tenemos intención de voto para todos los casos. 

df_all['P21'].value_counts()

C:\Users\Victor\AppData\Local\Temp\ipykernel_10388\1516723314.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred['P21'] = y_predicted


NO VOTARÁ      1123
PNV/EAJ         790
EH BILDU        677
PSE-EE          219
PP               67
VOTO BLANCO      58
PODEMOS          28
SUMAR            27
VOX              15
VOTO NULO        13
OTRO             13
Name: P21, dtype: int64

In [18]:
# Pues ya estaría, ¿no?, ya tenemos la estimación de voto.
 #Pues no. Tenemos lo que los encuestados nos dicen que van a votar, los 3030 encuestados, que ya es (antes de nuestro 
 #modelo de ML sólo lo teníamos para 1810 casos, ya que había 1220 'NS/NC'). Vaya, tenemos la intención de voto. Pero la 
 #estimación de voto es algo más compleja. ¿Por qué? Porque hay gente que dice a quién va a votar pero finalmente no vota.
 #Y porque hay gente que miente sobre a quién va a votar. ¿Y tenemos manera de saber quién nos miente? No exactamente
 #quién, sino en qué medida nos están mintiendo para cada una de las opciones de voto. Y en función de la medida en que
 #nos están mintiendo, podemos corregir (ponderando) el voto a esas opciones.

# Venga, vamos a ello, que mola bastante. Ya estamos casi terminandooo!

# Lo primero, la gente que dice que va a votar pero termina por no votar. Tenemos una variable que nos da alguna
 #pista al respecto, la 'P20' es una pregunta sobre la probabilidad de ir a votar. Pues bien, vamos a quedarnos con aquellos
 #que responden que acudirán a votar 'Sí, seguro' y 'Probablemente sí', que se correspoden con los códigos de respuesta 1 y
 #2 respectivamente. Así nuestra estimación de voto incluirá sólo a aquellos que tienen más probabilidad de ir a votar,
 #dejando fuera a aquellos que probablemente no votarán. Total, que hay que filtrar nustro df.

df_all = df_all[(df_all['P20'] == 1) | (df_all['P20'] == 2)]
print(f'Número de casos de df_all: {len(df_all)}')

# El número de casos se ha reducido significativamente, de 3030 a 2504, nos hemos quitado un buen numero de abstencionistas.

Número de casos de df_all: 2504


In [19]:
# Y ahora a poner en funcionamiento el detector de mentiras. Decíamos antes que lo único que podemos saber al respecto es 
 #la medida en que nos están mintiendo, en conjunto, para cada una de las opciones de voto. ¿Y cómo podemos saberlo?
 #Pues con el único dato real y cuantificable que tenemos, los resultados (reales) de las pasadas elecciones. Y comparando 
 #ese porcentaje real de voto con el recuerdo de voto de las pasadas elecciones ('P23') de la encuesta observaremos 
 #infrarecuerdos y sobrerecuerdos de voto que nos permitirán ponderar los casos. Y ya tendríamos nuestra estimación.

# Pero para, para. Vamos por pasos. Lo primero que necesitamos es el cruce entre la intención de voto ('P21') y el recuerdo
 #de voto de las pasadas elecciones ('P23'). Vaya, una tabla de contingencia.

vote_table = pd.crosstab(df_all['P21'], df_all['P23'], margins = True, margins_name="Total")

print(vote_table)

# Hecho, ya estamos para terminar. Lo haremos sobre excel, por dos razones. Una, soy un nostálgico y siempre he rematado las
 #estimaciones de voto en excel. Dos, creo que objetivamente es más sencillo y más visual.

# Así que nos llevamos esa tabla de contingencia a excel.

vote_table.to_excel('vote_table.xlsx', index=True)

P23          EH BILDU  NO VOTÓ  NS/NC  OTRO  PNV/EAJ  PODEMOS  \
P21                                                             
EH BILDU          541       10     25     1       39       42   
NO VOTARÁ          20       58    401     5       87       31   
OTRO                0        1      2     4        0        0   
PNV/EAJ             5       22     38     0      687        1   
PODEMOS             2        0      2     1        0       20   
PP                  1        4      6     0        9        0   
PSE-EE              0        6      9     1        8        7   
SUMAR               1        2      3     1        1       16   
VOTO BLANCO         1        0     14     1        9        1   
VOTO NULO           0        2      0     2        2        0   
VOX                 0        1      0     0        2        1   
Total             571      106    500    16      844      119   

P23          PP + CIUDADANOS  PSE-EE  VOTÓ EN BLANCO  VOTÓ NULO  VOX  Total  
P21        